In [202]:
from __future__ import division
import os,glob,random
import cv2
from sklearn.model_selection import train_test_split
from keras.models import load_model
import numpy as np
from numpy import expand_dims
from sklearn import preprocessing
from mtcnn.mtcnn import MTCNN
from cv2 import CascadeClassifier

#print(tf.test.gpu_device_name())
model = load_model('./keras-facenet/model/facenet_keras.h5')
print('Loaded Facenet Model')
print('Loaded MTCNN Face Detector')

Loaded Facenet Model
Loaded MTCNN Face Detector


D:\Conda Installation\envs\tf-gpu\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [203]:
np.random.seed(122)
data = []
all_labels = []
X_train = []
labels_train = []
X_val = []
labels_val = []
path = "D:/Desktop/Facial Recognition"
print ("Check if path exists", os.path.exists(path))


# Load all images in the lists
for root, dirs,files in os.walk('.'):
    #print(dirs)
    for directory in dirs:
        labels_path =  os.path.join(root, directory)
        pics = [name for name in glob.glob(os.path.join(labels_path, '*.jpg'))]
        pics.extend([name for name in glob.glob(os.path.join(labels_path, '*.jfif'))])
        if not pics==[]:
            train , val  = train_test_split(pics, test_size = 0.2)
            X_train.extend(train) 
            X_val.extend(val)
            l_val = [directory for i in range(len(val))] #these labels correspond to one class for testing
            l_train = [directory for i in range(len(train))] #these labels for one class for training
            labels_train.extend(l_train) #Append these labels to the whole list that holds all the classes
            labels_val.extend(l_val) #Append the labels to the whole list of class labels
            all_labels.append(directory) #add all classes to a list-->later used for encoding
            
            
# Print all classes/identities
print(all_labels)
print("The validation labels are: ", labels_val, "\n")
print("The training labels are: ", labels_train, "\n")



Check if path exists True
['Kara', 'Mitsotakis', 'Will']
The validation labels are:  ['Kara', 'Kara', 'Kara', 'Kara', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Will', 'Will', 'Will'] 

The training labels are:  ['Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Kara', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Mitsotakis', 'Will', 'Will', 'Will', 'Will', 'Will', 'Will', 'Will', 'Will', 'Will', 'Will', 'Will'] 



In [204]:
#Methods used for FACE RECOGNITION

# Exract a detected face
mtcnn = MTCNN()
#or
#classifier = CascadeClassifier('adaboost_cascade.txt')
def extract_face(filename , Isfilename = True ):
    size = (160,160) #required size for MTCNN
    if (Isfilename): 
        image = cv2.imread(filename)
    else: 
        image =filename
    print(filename)
    bboxes = mtcnn.detect_faces(image) #uncomment to use MTCNN
    #bboxes = classifier.detectMultiScale(image, 1.20 ,3) #uncomment for Viola Jones Cascade classifier
    try:
        for box in bboxes:
            left_corner_x , left_corner_y , width, height = box['box'] #Uncomment when using the MTCNN
            #left_corner_x , left_corner_y , width, height = box #Uncomment when using the Viola_Jones Cascade Classifier
            x_stop , y_stop =  left_corner_x+width , left_corner_y+height
        face = image[left_corner_y:y_stop, left_corner_x: x_stop]
        face_rsz = cv2.resize(face,size)
        return face_rsz , {left_corner_x, left_corner_y , x_stop, y_stop}
    except:
        raise Exception("The picture under the filename "+filename+" has no detected faces. Consider removing this picture")
        
    

# Get embedding of extracted face
def get_embedding(model, face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    print(samples.shape)
    # make prediction to get embedding
    yhat = model.predict(samples)
    return yhat[0]

In [205]:
# convert each face in the train set to an embedding
emTrainX = list()
for pic in X_train:
    face_pixels, original_pixels  = extract_face(pic, True) #get the pixels of the extracted face
    print(face_pixels.shape)
    embedding = get_embedding(model, face_pixels) #get embedding (lower dimensional representation)
    emTrainX.append(embedding) #the train array holds all the embeddings corresponding to each face
emTrainX = np.asarray(emTrainX)
np.savez_compressed('my_dataset_embeddings.npz', emTrainX, labels_train)


# convert each face in the val set to an embedding
emValX = list()
for pic in X_val:
    face_pixels , original_pixels = extract_face(pic , True)
    embedding = get_embedding(model, face_pixels)
    emValX.append(embedding)
emValX =np.asarray(emValX)


.\Kara\Giorgos-Karagounis-008.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\Georgios+Karagounis+Greece+v+Russia+International+a_O335G2R5Al.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\8247.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\giorgos-karagounis-755e7c5d-552a-4591-beaa-e7e11e369da-resize-750.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\kara.jfif
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\giorgos-karagounis.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\Giorgos+Karagounis+Greece+v+Czech+Republic+Kc421UmP6E-l.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\images.jfif
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\Giorgos+Karagounis+Poland+v+Greece+Group+UEFA+3_ftraqH_vEl.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\717xj1Zp7dL._SY606_.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\giorgoskaragounis_20100612.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\download.jfif
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\download (1).jfif
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\karagounisasa.jpg
(160, 160, 3)
(1, 160, 160, 3)
.\Kara\giorgos-karagou

In [199]:
def correct (pred,truth ):
    if (pred == truth):
        return "True"
    else: return "False"

In [200]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

# normalize input vectors
normalizer = Normalizer(norm='l2')
X_train = normalizer.transform(emTrainX)
X_val = normalizer.transform(emValX)
# label encode targets
label_encoder = LabelEncoder()
label_encoder.fit(labels_train)
y_train = label_encoder.transform(labels_train)
y_val = label_encoder.transform(labels_val)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)
pred_index = model.predict(X_val)
predict_prob = model.predict_proba(X_val)
#Convert class index to Class names/identities
pred_class = label_encoder.inverse_transform(pred_index)
full_preds = []
for sample in range(0, len(pred_index)):
    new_pred = [" %s (%.4f) /n %s" % (pred_class[sample] , predict_prob[sample,pred_index[sample]],correct(pred_class[sample], labels_val[sample]))] #this will be the prediction for the specific sample
    full_preds.extend(new_pred)  
print(full_preds)
print(predict_prob)

[' Kara (0.8415) /n True', ' Kara (0.9419) /n True', ' Kara (0.7592) /n True', ' Kara (0.8560) /n True', ' Mitsotakis (0.8446) /n True', ' Mitsotakis (0.9204) /n True', ' Mitsotakis (0.8819) /n True', ' Mitsotakis (0.8607) /n True', ' Will (0.8984) /n True', ' Will (0.8764) /n True', ' Will (0.8365) /n True']
[[0.84149725 0.07628476 0.08221799]
 [0.94185841 0.03569378 0.02244781]
 [0.75924251 0.12362382 0.11713368]
 [0.85595423 0.04934833 0.09469744]
 [0.09883533 0.84457018 0.05659448]
 [0.05683408 0.92035484 0.02281108]
 [0.07343289 0.88190777 0.04465934]
 [0.11205156 0.86066073 0.0272877 ]
 [0.05012771 0.05146104 0.89841125]
 [0.07750919 0.04609942 0.87639139]
 [0.08139846 0.08208107 0.83652047]]
